In [58]:
import numpy as np
from datetime import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [71]:
url = "http://vps-40d69db1.vps.ovh.ca:22001/api/ExecucaoSimulacaos/melhorValidacaoPeriodoFlat?idPeriodo=9"

In [72]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [61]:
df.columns

Index(['simulacaoId', 'ticker', 'simulacaoResultado', 'simulacaoLucro',
       'simulacaoPrejuizo', 'target', 'stop', 'tipo', 'experimentoSimulacaoId',
       'validacaoLucro', 'validacaoPrejuizo', 'saldoValidacao'],
      dtype='object')

Qual o período ?
Quantos experimentos ?
Total de casos ?

In [62]:
dfResumo = df.groupby('experimentoSimulacaoId').count()
dfResumo = dfResumo.filter(items=['ticker', 'validacaoLucro','saldoValidacao'])
dfResumo['percentual'] = (dfResumo['validacaoLucro'] / dfResumo['ticker']) * 100
dfResumo.head()

,ticker,validacaoLucro,saldoValidacao,percentual
experimentoSimulacaoId,,,,
54,14458,14422,14422,99.751003
56,11339,11329,11329,99.911809
57,27537,27528,27528,99.967317
58,13303,8965,8965,67.390814


Tickers 

In [63]:
print("Quantidade total de tickers: %2d "%(len(df['ticker'].unique())))

Quantidade total de tickers: 60 


In [64]:
dfTicker = df.groupby(['ticker','saldoValidacao']).size().reset_index(name ='qtde')

In [65]:
dfTicker.head()

,ticker,saldoValidacao,qtde
0,AERI3,0.0,227
1,AERI3,1.0,22
2,AERI3,2.0,50
3,ALPA4,0.0,695
4,ALSO3,0.0,21


In [66]:
print("Quantidade de tickers já tratados: %2d "%(len(dfTicker['ticker'].unique())))

Quantidade de tickers já tratados: 56 


In [67]:
dfTicker2 = dfTicker.pivot_table('qtde','ticker','saldoValidacao')
dfTicker2.fillna(0, inplace=True)
dfTicker2['positivo'] = dfTicker2[1.0] + dfTicker2[2.0]
dfTicker2['negativo'] = dfTicker2[-1.0] + dfTicker2[-2.0] + dfTicker2[-3.0]
dfTicker2.drop(columns=[-3.0, -2.0, -1.0, 1.0, 2.0], inplace=True)
dfTicker2.head(11)

saldoValidacao,0.0,positivo,negativo
ticker,,,
AERI3,227.0,72.0,0.0
ALPA4,695.0,0.0,0.0
ALSO3,21.0,120.0,0.0
AMAR3,658.0,155.0,261.0
ANIM3,364.0,68.0,1182.0
AZUL4,240.0,72.0,0.0
B3SA3,3300.0,405.0,0.0
BKBR3,705.0,1107.0,0.0
BPAC11,5371.0,0.0,2217.0


In [68]:

dfBom = dfTicker2[(dfTicker2['negativo']==0) & (dfTicker2['positivo']>0)]

In [69]:
len(dfBom)

18

In [70]:
dfBom.head(25)

saldoValidacao,0.0,positivo,negativo
ticker,,,
AERI3,227.0,72.0,0.0
ALSO3,21.0,120.0,0.0
AZUL4,240.0,72.0,0.0
B3SA3,3300.0,405.0,0.0
BKBR3,705.0,1107.0,0.0
BPAN4,2283.0,157.0,0.0
BRAP4,0.0,82.0,0.0
BRML3,2280.0,72.0,0.0
CVCB3,0.0,252.0,0.0


# Relacionar isso com os dados de regra-projecao.

### <<dataframe>>resultadoTickerComRegraProjecao(in regraProjecao, in dataInicio)

In [73]:
url2 = "http://vps-40d69db1.vps.ovh.ca:22001/api/RegraProjecaos/tickerPerformance?dataNumInicio=20210901&dataNumFinal=20211030&grupoAcao=Neg-25-02&codigoRegra=1"

In [79]:
r = requests.get(url2)
j = r.json()
df2 = pd.DataFrame.from_dict(j)
df2.set_index('ticker', inplace=True)

In [80]:
df2.head()

,ticks,positivos,negativos,zero
ticker,,,,
BBSE3,615,416,0,199
BRFS3,615,215,317,83
CCRO3,615,77,0,538
CIEL3,615,105,219,291
CMIG4,615,20,32,563


In [84]:
dfjoin = df2.join(dfTicker2)

In [85]:
dfjoin.sort_values(by = 'negativos', ascending = False)

,ticks,positivos,negativos,zero,0.0,positivo,negativo
ticker,,,,,,,
LWSA3,615,17,598,0,847.0,2082.0,286.0
CYRE3,615,51,519,45,869.0,0.0,450.0
HAPV3,615,123,492,0,44.0,53.0,0.0
GOAU4,615,47,481,87,NaN,NaN,NaN
GOLL4,615,152,463,0,388.0,0.0,318.0
RENT3,615,146,457,12,1256.0,72.0,445.0
COGN3,615,125,447,43,1913.0,17.0,351.0
TOTS3,615,15,427,173,88.0,60.0,0.0
UGPA3,615,109,371,135,366.0,0.0,462.0
